In [1]:
%reset -f

In [2]:
import openpyxl
from openpyxl.utils import column_index_from_string
import pandas as pd
import importlib
import simulation_class  

importlib.reload(simulation_class)
HeatPumpSimulator = simulation_class.HeatPumpSimulator


## Extraction of the Output Data

Extraction of output data from the `All data` sheet of the Excel file and storing in a DataFrame `df_output`.

- Column K : Water out -> water temperature leaving the heat pump; FMU output
- Column O : Power out -> thermal power of the heat pump (hot side); FMU output
- Column P : Power in -> electrical power of the heat pump; FMU output
- Column Q : COP -> coefficient of performance of the heat pump; FMU output

In [3]:
excel_file_path = './../data/Databrut.xlsx'

workbook = openpyxl.load_workbook(excel_file_path)
sheet = workbook.active

# Définir les colonnes et les noms correspondants
columns_info = {
    "K": "Water_out",
    "O": "Power_out",
    "P": "Power_in",
    "Q": "COP",
    "M": "Defrosts"
}

stored_data = {name: [] for name in columns_info.values()}

# Itérer sur les colonnes et stocker les données dans le dictionnaire
for col_letter, col_name in columns_info.items():
    col_index = column_index_from_string(col_letter)
    if col_index <= sheet.max_column:
        for cell in sheet[col_letter]:
            stored_data[col_name].append(cell.value)
    else:
        print(f"Column {col_letter} not found in the sheet.")

workbook.close()

df_output = pd.DataFrame(stored_data)
df_output = df_output.drop([0, 1, 2, 3])  

for column in ["Water_out", "Power_out", "Power_in", "COP"]:
    df_output[column] = pd.to_numeric(df_output[column], errors='coerce')

df_output.dropna(subset=["Water_out", "Power_out", "Power_in", "COP"], inplace=True)
df_output['Water_out'] = df_output['Water_out'] + 273.15
df_output = df_output.rename(columns={'Water_out': 'y_T_out', 'Power_out': 'y_heatPower', 'Power_in': 'y_elecPower', 'COP': 'y_COP'})

df_output.head()


,y_T_out,y_heatPower,y_elecPower,y_COP,Defrosts
4,308.248729,8628.775284,2022.298343,4.279641,no
5,308.320939,8589.444308,2044.149171,4.214619,no
6,308.179724,7641.797307,1748.723757,4.383122,no
7,308.268453,6873.783742,1477.259669,4.667061,no
8,308.200442,6035.440249,1248.497238,4.848746,no


In [4]:
#Export l'output en csv
Outputpath = './../data/df_output.csv'
df_output.to_csv(Outputpath, index=False)

## Extraction of the Input Data

Extraction of the input data from the `All data` sheet of the Excel file and storing in a DataFrame `df_input`.

- Column I : Frequency -> compressor frequency; FMU input
- Column J : Water in -> water temperature at the heat pump inlet; FMU input
- Column L : Flow -> water flow; FMU input
- Column M : Defrosts -> indicates if defrosting took place during the test

In [5]:
excel_file_path = './../data/Databrut.xlsx'

workbook = openpyxl.load_workbook(excel_file_path)
sheet = workbook.active

columns_info = {
    "I": "Frequency",
    "J": "Water_in",
    "L": "Flow",
    "M": "Defrosts"  
}

stored_data = {name: [] for name in columns_info.values()}

for col_letter, col_name in columns_info.items():
    col_index = column_index_from_string(col_letter)
    if col_index <= sheet.max_column:
        for row in sheet.iter_rows(min_row=2, max_col=col_index, min_col=col_index, values_only=True):  
            stored_data[col_name].extend(row)
    else:
        print(f"Column {col_letter} not found in the sheet.")

workbook.close()

df_input = pd.DataFrame(stored_data)

for column in ["Frequency", "Water_in", "Flow"]:
    df_input[column] = pd.to_numeric(df_input[column], errors='coerce')

df_input.dropna(subset=["Frequency", "Water_in", "Flow"], inplace=True)


# Création de la colonne 'T_air' avec les valeurs correctes
NbA12W35,NbA12W45 = 9,9
NbA7W35,NbA7W45,NbA7W55 = 13,12,4
NbA2W35,NbA2W45,NbA2W55  = 5,5,5
NbAmoins7W35,NbAmoins7W45,NbAmoins7W55 = 5,5,5

NbA12 = NbA12W35 + NbA12W45
NbA7 = NbA7W35 + NbA7W45 + NbA7W55
NbA2 = NbA2W35 + NbA2W45 + NbA2W55
NbAmoins7 = NbAmoins7W35 + NbAmoins7W45 + NbAmoins7W55

values = [12]*NbA12 + [7]*NbA7 + [2]*NbA2 + [-7]*NbAmoins7

if len(values) == len(df_input):
    df_input['T_air'] = values
else:
    print("Erreur : La longueur des valeurs de 'T_air' ne correspond pas au DataFrame.")

columns_order = ["Frequency", "Water_in", "Flow", "T_air", "Defrosts"] 
df_input = df_input[columns_order]  # Réorganisation des colonnes

# df_input.head()


In [6]:
# Conversion de la température de l'eau de °C à K
df_input['Water_in'] = df_input['Water_in'] + 273.15

# Conversion du débit d'eau de m³/h à m³/s
df_input['Flow'] = df_input['Flow'] / 3600

# Conversion de la température de l'air de °C à K
df_input['T_air'] = df_input['T_air'] + 273.15

df_input.head()

,Frequency,Water_in,Flow,T_air,Defrosts
3,105.6,303.262818,0.000415,285.15,no
4,98.4,303.289006,0.000409,285.15,no
5,88.2,303.231215,0.000370,285.15,no
6,78.0,303.359724,0.000336,285.15,no
7,68.4,303.246077,0.000292,285.15,no


In [7]:
# export l'input en fichier Excel
Inputpath = './../data/df_input.csv'
df_input.to_csv(Inputpath, index=False)


## Calibration (defrost = no) 

Firstly, we ignore the tests where "Defrosts" = yes (column M)

In [8]:
df_no_defrosts = df_input[df_input['Defrosts'] == 'no']

In this notebook, we are importing the Python class `HeatPumpSimulator` that utilizes FMpy to simulate FMU. This class is responsible for performing the simulation of the heat pump system. Below are the defined inputs and outputs for the simulation.


#### FMU Inputs

- **u_compressorFrequency:** Compressor Frequency *Unit:* Hz

- **u_VFlow:** Water Flow *Unit:* m³/s

- **u_T_water:** Water Temperature Entering the Heat Pump *Unit:* K

- **u_T_air:** Air Temperature at the Heat Pump Inlet *Unit:* K

#### FMU Outputs

- **y_T_out:** Water Temperature Leaving the Heat Pump *Unit:* K

- **y_heatPower:** Thermal Power of the Heat Pump (Hot Side) *Unit:* W

- **y_elecPower:** Electrical Power of the Heat Pump *Unit:* W

- **y_COP:** Coefficient of Performance of the Heat Pump *Unit:* []


In [9]:
simulator = HeatPumpSimulator('../FMU/HPFMU_20_Linux.fmu')

last_rows = []

# Itération sur les 10 premières lignes du DataFrame d'entrée
for index, row in df_no_defrosts.head(10).iterrows():
    start_values = {
        'u_compressorFrequency': row['Frequency'],
        'u_VFlow': row['Flow'],
        'u_T_watrer_in': row['Water_in'],
        'u_T_air': row['T_air'],
        # Valeurs fixes des paramètres pour l'instant
        'x_areaLeakage': 5e-7,
        'x_areaSuctionValve': 5e-4,
        'x_areaDischargeValve': 5e-5,
        'x_relativeDeadSpace': 0.05,
        'x_driveEfficiency': 0.9
    }
    # print(f"start_values={start_values}")

    simulation_result = simulator.simulate(start_values)
    df_simulation_result = pd.DataFrame(simulation_result)
    last_row = df_simulation_result.iloc[-1]
    last_rows.append(last_row)


start_values={'u_compressorFrequency': 105.6, 'u_VFlow': 0.0004146853898097, 'u_T_watrer_in': 303.262817679558, 'u_T_air': 285.15, 'x_areaLeakage': 5e-07, 'x_areaSuctionValve': 0.0005, 'x_areaDischargeValve': 5e-05, 'x_relativeDeadSpace': 0.05, 'x_driveEfficiency': 0.9}
start_values={'u_compressorFrequency': 98.4, 'u_VFlow': 0.0004090147329650083, 'u_T_watrer_in': 303.28900552486186, 'u_T_air': 285.15, 'x_areaLeakage': 5e-07, 'x_areaSuctionValve': 0.0005, 'x_areaDischargeValve': 5e-05, 'x_relativeDeadSpace': 0.05, 'x_driveEfficiency': 0.9}
start_values={'u_compressorFrequency': 88.2, 'u_VFlow': 0.0003700322283609583, 'u_T_watrer_in': 303.2312154696133, 'u_T_air': 285.15, 'x_areaLeakage': 5e-07, 'x_areaSuctionValve': 0.0005, 'x_areaDischargeValve': 5e-05, 'x_relativeDeadSpace': 0.05, 'x_driveEfficiency': 0.9}
start_values={'u_compressorFrequency': 78.0, 'u_VFlow': 0.0003355325352977278, 'u_T_watrer_in': 303.359723756906, 'u_T_air': 285.15, 'x_areaLeakage': 5e-07, 'x_areaSuctionValve': 0

In [10]:
df_final_results = pd.DataFrame(last_rows)
df_final_results = df_final_results.drop(columns=['time'])
df_final_results = df_final_results.reset_index(drop=True)

df_final_results.head()


,y_T_out,y_heatPower,y_elecPower,y_COP
0,308.585764,9181.373552,2427.614040,3.782057
1,308.320366,8559.788081,2223.600652,3.849517
2,308.191957,7635.408713,1945.937162,3.923769
3,308.135888,6669.292055,1680.123506,3.969525
4,307.972277,5738.200241,1435.660306,3.996907
